In [ ]:
# this script is for the calculation of PDFs in Figure 2 of Rodgers et al. 2021 (https://doi.org/10.5194/esd-2021-50). 
# If you have have any questions, please contact the author of this notebook.
# Author: Lei Huang (huanglei[AT]pusan[DOT]ac[DOT]kr)

# import

In [64]:
%matplotlib inline

import numpy as np
import xarray as xr 
import matplotlib.pyplot as plt
import matplotlib as mpl
import glob
import dask.array as da
import pandas as pd


# seting parallel

In [ ]:
## Run the mpirun in command line:
## mpirun --np 6 dask-mpi --scheduler-file scheduler.json --no-nanny --dashboard-address :8785 --memory-limit=60e9

from dask.distributed import Client
client = Client(scheduler_file = 'the_path_for_your_scheduler_json_file')

# functions for reading ensembles in parallel

In [66]:
# preprocess dataset prior to concatenation
variables = []
exceptcv = ['time', 'nlat', 'nlon', 'z_t',
            'lon', 'lat', 'gw', 'landfrac', 'area', *variables]
def def_process_coords(exceptcv = []):
    def process_coords(ds, except_coord_vars=exceptcv):
        coord_vars = []
        for v in np.array(ds.coords):
            if not v in except_coord_vars:
                coord_vars += [v]
        for v in np.array(ds.data_vars):
            if not v in except_coord_vars:
                coord_vars += [v]
        return ds.drop(coord_vars)
    return process_coords



In [67]:
# define function to read in files for historical simulations
def read_in(var, exceptcv, domain='lnd/', freq='day_1/', stream='h6', chunks=dict(time=365), ens_s = 0, ens_e = 100):
    ens_dir = "mother_directory_for_ensemble_files"
    projens_names = [member.split('archive/')[1][:-1] for member in sorted(
        glob.glob(ens_dir + "b.e21.BSSP370*.f09_g17*/"))][ens_s:ens_e]
    proj_ncfiles = []
    for i in np.arange(len(projens_names)):
        proj_fnames = sorted(glob.glob(
            ens_dir + projens_names[i] + "/" + domain + "proc/tseries/" + freq + "*" + stream + var + "*"))
        proj_ncfiles.append(proj_fnames[-2:])
    ens_numbers = [members.split('LE2-')[1]
                   for members in projens_names]
    proj_ds = xr.open_mfdataset(proj_ncfiles,
                                chunks=chunks,
                                preprocess=def_process_coords(exceptcv),
                                combine='nested',
                                concat_dim=[[*ens_numbers], 'time'],
                                parallel=True)

    ens_ds = proj_ds.rename({'concat_dim': 'ensemble'})
    return ens_ds


# PDF for Nino3.4 SST

In [6]:
# read in SST for period of 1980-1989
variables = ['SST']
exceptcv = ['time', 'nlat', 'nlon', 'z_t', 'TAREA', *variables]
ncfiles = sorted(glob.glob('mother_directory_for_ensemble_files/b.e21.BHISTcmip6*/ocn/proc/tseries/day_1/*.SST.1980*')) \
    + sorted(glob.glob('mother_directory_for_ensemble_files/b.e21.BHISTsmbb*/ocn/proc/tseries/day_1/*.SST.1980*'))
hist_ens_numbers = [member.split('LE2-')[1][:8] for member in ncfiles]
sst_hist_ds = xr.open_mfdataset(ncfiles,
                          chunks={'time':365},
                          combine='nested',
                                preprocess=def_process_coords(exceptcv),
                          concat_dim =[[*hist_ens_numbers]],
                          parallel = True).rename({'concat_dim':'ensemble'})


In [32]:
# read in SST for period of 2090-2099
sst_proj_ds = read_in(var = '.SST.',
                     exceptcv = exceptcv,
                     domain = 'ocn/',
                     freq = 'day_1/',
                     stream = 'h*',
                     chunks= dict(time = 365))


In [59]:
# select the regions for Nino3
sst_hist_nino = sst_hist_ds.SST[:,:,...].sel(nlat = slice(168,206), nlon = slice(204,249))
sst_proj_nino = sst_proj_ds.SST.sel(nlat = slice(168,206), nlon = slice(204,249), time = slice('2090-01-02','2100-01-01'))
# tarea is the cell area on the T-grid of POP2
tarea_hist_nino = sst_hist_ds.TAREA.sel(nlat = slice(168,206), nlon = slice(204,249)).broadcast_like(sst_hist_nino).chunk({'time':sst_hist_nino.chunks[1]})
tarea_proj_nino = sst_proj_ds.TAREA.sel(nlat = slice(168,206), nlon = slice(204,249),time = slice('2090-01-02','2100-01-01')).broadcast_like(sst_proj_nino).chunk({'time':sst_proj_nino.chunks[1]})

In [69]:
# calculate the PDF for SST in 1980-1989
# please refer to the document of dask.array.histogram for more information
h_hist_sst_nino_raw, bins_hist_sst_nino_raw = da.histogram(sst_hist_nino,
                                                          bins = np.arange(15,40.2,0.2),
                                                          weights = tarea_hist_nino,
                                                          density = True)
h_hist_sst_nino_raw = h_hist_sst_nino_raw.compute()

In [70]:
# calculate the PDF for SST in 2090-2099
h_proj_sst_nino_raw, bins_proj_sst_nino_raw = da.histogram(sst_proj_nino,
                                                          bins = np.arange(15,40.2,0.2),
                                                          weights = tarea_proj_nino,
                                                          density = True)
h_proj_sst_nino_raw = h_proj_sst_nino_raw.compute()


In [147]:
# save the result to csv file
s1 = np.expand_dims(bins_hist_sst_nino_raw[1:]-0.1, axis = 1)
s2 = np.expand_dims(h_hist_sst_nino_raw, axis = 1)
s3 = np.expand_dims(h_proj_sst_nino_raw, axis = 1)
pd.DataFrame(data = np.concatenate((s1,s2,s3), axis = 1),
            columns= ['bins', 'h_hist', 'h_proj']).to_csv('path_csv_file', index = False)

# Fire counts in California

In [318]:
variables = ['NFIRE']
exceptcv = ['time', 'lat', 'lon', 'landfrac', 'area',  *variables]
ncfiles = sorted(glob.glob('mother_directory_for_ensemble_files/b.e21.BHISTcmip6*/lnd/proc/tseries/day_1/*.NFIRE.1980*')) \
    + sorted(glob.glob('mother_directory_for_ensemble_files/b.e21.BHISTsmbb*/lnd/proc/tseries/day_1/*.NFIRE.1980*'))
hist_ens_numbers = [member.split('LE2-')[1][:8] for member in ncfiles]
nfire_hist_ds = xr.open_mfdataset(ncfiles,
                          chunks={'time':365},
                          combine='nested',
                                  preprocess=def_process_coords(exceptcv),
                          concat_dim =[[*hist_ens_numbers]],
                          parallel = True).rename({'concat_dim':'ensemble'})


In [320]:
nfire_proj_ds = read_in(var = '.NFIRE.',
                     exceptcv = exceptcv,
                     domain = 'lnd/',
                     freq = 'day_1/',
                     stream = 'h5',
                     chunks= dict(time = 365),
                     ens_s = 10,
                    ens_e = 100)

In [321]:
nfire_hist_calif = nfire_hist_ds.NFIRE.sel(lat = slice(32,41), lon = slice(235,242))*10000*365*24*3600 # convert the unit to the one shown in Figure 2
nfire_proj_calif = nfire_proj_ds.NFIRE.sel(lat = slice(32,41), lon = slice(235,242), time = slice('2090-01-01','2099-12-31'))*10000*365*24*3600

In [322]:
landfrac_hist_calif = nfire_hist_ds.landfrac.sel(lat = slice(32,41), lon = slice(235,242))
landfrac_proj_calif = nfire_proj_ds.landfrac.sel(lat = slice(32,41), lon = slice(235,242), time = slice('2090-01-01','2099-12-31'))
area_hist_calif = nfire_hist_ds.area.sel(lat = slice(32,41), lon = slice(235,242))
area_proj_calif = nfire_proj_ds.area.sel(lat = slice(32,41), lon = slice(235,242), time = slice('2090-01-01','2099-12-31'))

In [323]:
landfrac_hist_calif = landfrac_hist_calif.broadcast_like(nfire_hist_calif).chunk({'time':nfire_hist_calif.chunks[1]})
landfrac_proj_calif = landfrac_proj_calif.broadcast_like(nfire_proj_calif).chunk({'time':nfire_proj_calif.chunks[1]})

In [324]:
area_hist_calif = area_hist_calif.broadcast_like(nfire_hist_calif).chunk({'time':nfire_hist_calif.chunks[1]})
area_proj_calif = area_proj_calif.broadcast_like(nfire_proj_calif).chunk({'time':nfire_proj_calif.chunks[1]})

In [325]:
nfire_hist_calif = nfire_hist_calif.where(landfrac_hist_calif >= 0.9)
nfire_proj_calif = nfire_proj_calif.where(landfrac_proj_calif >= 0.9)

In [331]:
h_hist_nfire_calif_raw, bins_hist_nfire_calif_raw = np.histogram(nfire_hist_calif,
                                                              bins = np.arange(0,2500.1,10),
                                                              weights = area_hist_calif,
                                                              density = True)

In [332]:
h_proj_nfire_calif_raw, bins_proj_nfire_calif_raw = np.histogram(nfire_proj_calif,
                                                              bins = np.arange(0,2500.1,10),
                                                              weights = area_proj_calif,
                                                              density = True)

In [334]:
s1 = np.expand_dims(bins_hist_nfire_calif_raw[1:] - 5, axis = 1)
s2 = np.expand_dims(h_hist_nfire_calif_raw, axis = 1)
s3 = np.expand_dims(h_proj_nfire_calif_raw, axis = 1)
pd.DataFrame(data = np.concatenate((s1,s2,s3), axis = 1),
             columns=['bins', 'h_hist', 'h_proj']).to_csv('path_csv_file', index=False)


# PDF for Chlorophyll in NA

In [86]:
# In the Biogeochemistry module, chlorophyll concentration equals the sum of diatChl_SURF, diazChl_SURF, and spChl_SURF
## read in chlorophyll for 1980-1989
variables = ['diatChl_SURF']
exceptcv = ['time', 'nlat', 'nlon', 'z_t', 'TAREA', *variables]
ncfiles = sorted(glob.glob('mother_directory_for_ensemble_files/b.e21.BHISTcmip6*/ocn/proc/tseries/day_1/*.diatChl_SURF.1980*')) \
            + sorted(glob.glob('mother_directory_for_ensemble_files/b.e21.BHISTsmbb*/ocn/proc/tseries/day_1/*.diatChl_SURF.1980*'))
hist_ens_numbers = [member.split('LE2-')[1][:8] for member in ncfiles]
tchl_hist_ds = xr.open_mfdataset(ncfiles,
                          chunks=dict(nlat = 192, nlon = 160, time = 365),
                          combine='nested',
                          preprocess=def_process_coords(exceptcv),
                          concat_dim =[[*hist_ens_numbers]],
                          parallel = True).rename({'concat_dim':'ensemble'})
ncfiles = sorted(glob.glob('mother_directory_for_ensemble_files/b.e21.BHISTcmip6*/ocn/proc/tseries/day_1/*.diazChl_SURF.1980*')) \
            + sorted(glob.glob('mother_directory_for_ensemble_files/b.e21.BHISTsmbb*/ocn/proc/tseries/day_1/*.diazChl_SURF.1980*'))
hist_ens_numbers = [member.split('LE2-')[1][:8] for member in ncfiles]
variables = ['diazChl_SURF']
exceptcv = ['time', 'nlat', 'nlon', 'z_t', 'TAREA', *variables]
zchl_hist_ds = xr.open_mfdataset(ncfiles,
                          chunks=dict(nlat = 192, nlon = 160, time = 365),
                          combine='nested',
                          preprocess=def_process_coords(exceptcv),
                          concat_dim =[[*hist_ens_numbers]],
                          parallel = True).rename({'concat_dim':'ensemble'})
ncfiles = sorted(glob.glob('mother_directory_for_ensemble_files/b.e21.BHISTcmip6*/ocn/proc/tseries/day_1/*.spChl_SURF.1980*')) \
            + sorted(glob.glob('mother_directory_for_ensemble_files/b.e21.BHISTsmbb*/ocn/proc/tseries/day_1/*.spChl_SURF.1980*'))
hist_ens_numbers = [member.split('LE2-')[1][:8] for member in ncfiles]
variables = ['spChl_SURF']
exceptcv = ['time', 'nlat', 'nlon', 'z_t', 'TAREA', *variables]
spchl_hist_ds = xr.open_mfdataset(ncfiles,
                          chunks=dict(nlat = 192, nlon = 160, time = 365),
                          combine='nested',
                          preprocess=def_process_coords(exceptcv),
                          concat_dim =[[*hist_ens_numbers]],
                          parallel = True).rename({'concat_dim':'ensemble'})


In [87]:
## read in chlorophyll for 2090-2099
variables = ['diatChl_SURF']
exceptcv = ['time', 'nlat', 'nlon', 'z_t', 'TAREA', *variables]
tchl_proj_ds = read_in(var = '.diatChl_SURF.',
                     exceptcv = exceptcv,
                     domain = 'ocn/',
                     freq = 'day_1/',
                     stream = 'h*',
                     chunks= dict(nlat = 192, nlon = 160, time = 365),)
variables = ['diazChl_SURF']
exceptcv = ['time', 'nlat', 'nlon', 'z_t', 'TAREA', *variables]
zchl_proj_ds = read_in(var = '.diazChl_SURF.',
                     exceptcv = exceptcv,
                     domain = 'ocn/',
                     freq = 'day_1/',
                     stream = 'h*',
                     chunks= dict(nlat = 192, nlon = 160, time = 365),)
variables = ['spChl_SURF']
exceptcv = ['time', 'nlat', 'nlon', 'z_t', 'TAREA', *variables]
spchl_proj_ds = read_in(var = '.spChl_SURF.',
                     exceptcv = exceptcv,
                     domain = 'ocn/',
                     freq = 'day_1/',
                     stream = 'h*',
                     chunks= dict(nlat = 192, nlon = 160, time = 365),)


In [109]:
TLAT = xr.open_dataset(ncfiles[-1]).TLAT
TLONG = xr.open_dataset(ncfiles[-1]).TLONG

In [93]:
chl_hist = tchl_hist_ds.diatChl_SURF[:,:,...] \
            + zchl_hist_ds.diazChl_SURF[:,:,...] \
                + spchl_hist_ds.spChl_SURF[:,:,...]
chl_proj = tchl_proj_ds.diatChl_SURF.sel(time = slice('2090-01-02','2100-01-01')) \
            + zchl_proj_ds.diazChl_SURF.sel(time = slice('2090-01-02','2100-01-01')) \
            + spchl_proj_ds.spChl_SURF.sel(time = slice('2090-01-02','2100-01-01'))

In [97]:
tarea_hist = tchl_hist_ds.TAREA.broadcast_like(chl_hist).chunk({'time':chl_hist.chunks[1]})
tarea_proj = tchl_proj_ds.TAREA.sel(time = slice('2090-01-02','2100-01-01')).broadcast_like(chl_proj).chunk({'time':chl_proj.chunks[1]})

In [ ]:
chl_hist_NA = chl_hist.where((TLAT>=40) & (TLAT <= 60) & (TLONG >= 300) & (TLONG <= 345), drop = True)
chl_proj_NA = chl_proj.where((TLAT>=40) & (TLAT <= 60) & (TLONG >= 300) & (TLONG <= 345), drop = True)
tarea_hist_NA = tarea_hist.where((TLAT>=40) & (TLAT <= 60) & (TLONG >= 300) & (TLONG <= 345), drop = True)
tarea_proj_NA = tarea_proj.where((TLAT>=40) & (TLAT <= 60) & (TLONG >= 300) & (TLONG <= 345), drop = True)

In [133]:
h_hist_chl_NA_raw, bins_hist_chl_NA_raw = np.histogram(chl_hist_NA,
                                                      bins = np.arange(0,20.2,0.2),
                                                      weights = tarea_hist_NA,
                                                      density=True)

In [134]:
h_proj_chl_NA_raw, bins_proj_chl_NA_raw = np.histogram(chl_proj_NA,
                                                      bins = np.arange(0,20.2,0.2),
                                                      weights = tarea_proj_NA,
                                                      density=True)

In [148]:
s1 = np.expand_dims(bins_hist_chl_NA_raw[1:]-0.1, axis = 1)
s2 = np.expand_dims(h_hist_chl_NA_raw, axis = 1)
s3 = np.expand_dims(h_proj_chl_NA_raw, axis = 1)
pd.DataFrame(data = np.concatenate((s1,s2,s3), axis = 1), 
             columns=['bins', 'h_hist', 'h_proj']).to_csv('path_csv_file', index=False)


# NEP in Amazon

In [8]:
variables = ['NEP']
exceptcv = ['time', 'lat', 'lon', 'gw', 'landfrac', 'area',  *variables]
ncfiles = sorted(glob.glob('mother_directory_for_ensemble_files/b.e21.BHISTcmip6*/lnd/proc/tseries/day_1/*.NEP.1980*')) \
            + sorted(glob.glob('mother_directory_for_ensemble_files/b.e21.BHISTsmbb*/lnd/proc/tseries/day_1/*.NEP.1980*'))
hist_ens_numbers = [member.split('LE2-')[1][:8] for member in ncfiles]
nep_hist_ds = xr.open_mfdataset(ncfiles,
                          chunks={'time':365},
                          combine='nested',
                                preprocess=def_process_coords(exceptcv),
                          concat_dim =[[*hist_ens_numbers]],
                          parallel = True).rename({'concat_dim':'ensemble'})


In [9]:
nep_proj_ds = read_in(var = '.NEP.',
                     exceptcv = exceptcv,
                     domain = 'lnd/',
                     freq = 'day_1/',
                     stream = 'h5',
                     chunks= dict(time = 365),
                     ens_s = 10,
                    ens_e = 100)

In [301]:
nep_hist_amazon = nep_hist_ds.NEP.sel(lat = slice(-10,10), lon = slice(280,310))* 1000000
nep_proj_amazon = nep_proj_ds.NEP.sel(lat = slice(-10,10), lon = slice(280,310), time = slice('2090-01-01','2099-12-31'))* 1000000

In [305]:
landfrac_hist_amazon = nep_hist_ds.landfrac.sel(lat = slice(-10,10), lon = slice(280,310))
landfrac_proj_amazon = nep_proj_ds.landfrac.sel(lat = slice(-10,10), lon = slice(280,310), time = slice('2090-01-01','2099-12-31'))
area_hist_amazon = nep_hist_ds.area.sel(lat = slice(-10,10), lon = slice(280,310))
area_proj_amazon = nep_proj_ds.area.sel(lat = slice(-10,10), lon = slice(280,310), time = slice('2090-01-01','2099-12-31'))

In [307]:
area_hist_amazon = area_hist_amazon.broadcast_like(nep_hist_amazon).chunk({'time':nep_hist_amazon.chunks[1]})
area_proj_amazon = area_proj_amazon.broadcast_like(nep_proj_amazon).chunk({'time':nep_proj_amazon.chunks[1]})

In [308]:
nep_hist_amazon = nep_hist_amazon.where(landfrac_hist_amazon[0,...] >= 0.9)
nep_proj_amazon = nep_proj_amazon.where(landfrac_proj_amazon[0,...] >= 0.9)

In [313]:
h_hist_nep_amazon_raw, bins_hist_nep_amazon_raw = np.histogram(nep_hist_amazon,
                                                              bins = np.arange(-60,60.06,1),
                                                              weights = area_hist_amazon,
                                                              density = True)

In [314]:
h_proj_nep_amazon_raw, bins_proj_nep_amazon_raw = np.histogram(nep_proj_amazon,
                                                              bins = np.arange(-60,60.06,1),
                                                              weights = area_proj_amazon,
                                                              density = True)

In [315]:
s1 = np.expand_dims(bins_hist_nep_amazon_raw[1:] - 0.5, axis = 1)
s2 = np.expand_dims(h_hist_nep_amazon_raw, axis = 1)
s3 = np.expand_dims(h_proj_nep_amazon_raw, axis = 1)
pd.DataFrame(data = np.concatenate((s1,s2,s3), axis = 1),
             columns=['bins', 'h_hist', 'h_proj']).to_csv('path_csv_file', index=False)


# PDF for precipitation in Nino3.4

In [6]:
variables = ['PRECT']
exceptcv = ['time', 'lat', 'lon', 'gw',  *variables]
ncfiles = sorted(glob.glob('mother_directory_for_ensemble_files/b.e21.BHISTcmip6*/atm/proc/tseries/day_1/*.PRECT.1980*')) \
    + sorted(glob.glob('mother_directory_for_ensemble_files/b.e21.BHISTsmbb*/atm/proc/tseries/day_1/*.PRECT.1980*'))
hist_ens_numbers = [member.split('LE2-')[1][:8] for member in ncfiles]
prect_hist_ds = xr.open_mfdataset(ncfiles,
                          chunks={'time':365},
                          combine='nested',
                                  preprocess=def_process_coords(exceptcv),
                          concat_dim =[[*hist_ens_numbers]],
                          parallel = True).rename({'concat_dim':'ensemble'})


In [7]:
prect_proj_ds = read_in(var = '.PRECT.',
                     exceptcv = exceptcv,
                     domain = 'atm/',
                     freq = 'day_1/',
                     stream = 'h1',
                     chunks= dict(time = 365))

In [217]:
prect_hist_nino = prect_hist_ds.PRECT.sel(lat = slice(-5,5), lon = slice(190,240))* 24* 3600* 1000
prect_proj_nino = prect_proj_ds.PRECT.sel(lat = slice(-5,5), lon = slice(190,240), time = slice('2090-01-01','2099-12-31'))* 24* 3600* 1000

In [224]:
h_hist_prect_nino_raw, bins_hist_prect_nino_raw = np.histogram(prect_hist_nino,
                                                            bins = np.arange(0,1000.01,4),
                                                            weights = gw_hist_nino,
                                                            density = True)

In [225]:
h_proj_prect_nino_raw, bins_proj_prect_nino_raw = np.histogram(prect_proj_nino,
                                                            bins = np.arange(0,1000.01,4),
                                                            weights = gw_proj_nino,
                                                            density = True)

In [227]:
s1 = np.expand_dims(bins_hist_prect_nino_raw[1:] - 2, axis = 1)
s2 = np.expand_dims(h_hist_prect_nino_raw, axis = 1)
s3 = np.expand_dims(h_proj_prect_nino_raw, axis = 1)
pd.DataFrame(data = np.concatenate((s1,s2,s3), axis = 1),
             columns=['bins', 'h_hist', 'h_proj']).to_csv('path_csv_file', index=False)
